<a href="https://colab.research.google.com/github/nobuhikosekiya/elasticseach-query-apm-analyze-sample/blob/main/Elastic_Query_APM_analysis_auto_instrumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q elasticsearch elastic-apm
from pprint import pprint
import os, time
from getpass import getpass
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk
import elasticapm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.8/412.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 5.8 MB/s eta 0:00:00


# 接続先のElasitcsearchを設定

In [2]:
ELASTIC_CLOUD_ID = getpass("Elastic deployment Cloud ID")
ELASTIC_API_KEY = getpass("Elastic deployment API Key")
ELASTIC_URL = getpass("Elastic deployment Elasticsearch URL. No need if Cloud ID is provided.")
ELASTIC_USER = getpass("Elastic user. No need if API key is provided.")
ELASTIC_PASSWORD = getpass("Elastic password. No need if API key is provided.")


if ELASTIC_CLOUD_ID != '' and ELASTIC_API_KEY != '':
  es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER != '' and ELASTIC_PASSWORD != '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
    request_timeout=300
  )
elif ELASTIC_CLOUD_ID != '' and ELASTIC_USER != '' and ELASTIC_PASSWORD != '':
  es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER == '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    request_timeout=300
  )
else:
  print("env needs to set either ELASTIC_CLOUD_ID or ELASTIC_URL")


pprint(es.info()) # should return cluster info

Elastic deployment Cloud ID··········
Elastic deployment API Key··········
Elastic deployment Elasticsearch URL. No need if Cloud ID is provided.··········
Elastic user. No need if API key is provided.··········
Elastic password. No need if API key is provided.··········
ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': '5e6289c08b9e480daea7eda7cf903523', 'cluster_uuid': 'TNamOtbIQFG3BotZjeVenw', 'version': {'number': '8.11.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '76013fa76dcbf144c886990c6290715f5dc2ae20', 'build_date': '2023-12-05T10:03:47.729926671Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


# Initialize Python APM

In [3]:
ELASTIC_APM_URL = getpass("Elastic deployment APM URL.")
ELASITC_APM_SECRET= getpass("Elastic deployment APM secret token.")

client = elasticapm.Client(service_name="Elastic_Query_APM_analysis_auto-instrumentation", server_url=ELASTIC_APM_URL, secret_token=ELASITC_APM_SECRET, transaction_sample_rate=1.0)

# Need this for auto-instrumentation
elasticapm.instrument()

Elastic deployment APM URL.··········
Elastic deployment APM secret token.··········


# テストしよう

In [4]:
INDEX_NAME="Elastic_Query_APM_analysis_auto-instrumentation".lower()
INDEX_NAME

'elastic_query_apm_analysis_auto-instrumentation'

## INDEXの作成

In [5]:
if es.indices.exists(index=INDEX_NAME):
    # If it exists, delete the index
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' deleted successfully.")
else:
    print(f"Index '{INDEX_NAME}' does not exist.")

es.indices.create(
  index=INDEX_NAME,
  settings={
      "index": {
          "number_of_shards": 1,
          "number_of_replicas": 0
      }
  }
)

Index 'elastic_query_apm_analysis_auto-instrumentation' deleted successfully.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'elastic_query_apm_analysis_auto-instrumentation'})

## ドキュメントのインジェスト

In [6]:
docs_json=[
    {"question": "iphoneのインストール手順について教えて"},
    {"question": "I want to know how to setup an iphone"},
    {"question": "iosのセットアップ手順"},
    {"question": "iphoneどうやって使い始めればいいの？"},
    {"question": "androidのセットアップ方法などについて知りたい"},
    {"question": "iphoneかっこいいよね"},
    {"question": "iPhone 14が2月に発売されるらしい"}
]

# Delete previous documents
if es.indices.exists(index=INDEX_NAME):
    es.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})

index_docs = []
for doc_json in docs_json:
    index_docs.append({
        "_index": INDEX_NAME,
        "_source": doc_json,
    })

helpers.bulk(es, index_docs, refresh=True)

(7, [])

## サーチの実行 - 1

In [7]:
client.begin_transaction(transaction_type="app")
elasticapm.set_user_context(username="nobuhiko")

search_key_word="iPhone"
query={"match": {"question": search_key_word}}
response = es.search(index=INDEX_NAME, query=query)
print(response.body)

client.end_transaction(name="search-auto-instrument1", result="success")

{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 0.42356652, 'hits': [{'_index': 'elastic_query_apm_analysis_auto-instrumentation', '_id': 'kiQSY4wBoRDJc2lLPh5d', '_score': 0.42356652, '_source': {'question': 'iphoneかっこいいよね'}}, {'_index': 'elastic_query_apm_analysis_auto-instrumentation', '_id': 'jiQSY4wBoRDJc2lLPh5d', '_score': 0.40668815, '_source': {'question': 'I want to know how to setup an iphone'}}, {'_index': 'elastic_query_apm_analysis_auto-instrumentation', '_id': 'jSQSY4wBoRDJc2lLPh5d', '_score': 0.36326212, '_source': {'question': 'iphoneのインストール手順について教えて'}}, {'_index': 'elastic_query_apm_analysis_auto-instrumentation', '_id': 'kyQSY4wBoRDJc2lLPh5d', '_score': 0.33912128, '_source': {'question': 'iPhone 14が2月に発売されるらしい'}}, {'_index': 'elastic_query_apm_analysis_auto-instrumentation', '_id': 'kCQSY4wBoRDJc2lLPh5d', '_score': 0.32821545, '_source': {'question': 'ip

## サーチの実行 - 2

In [8]:
client.begin_transaction(transaction_type="app")
elasticapm.set_user_context(username="nobuhiko")

search_key_word="Windows"
query={"match": {"question": search_key_word}}
response = es.search(index=INDEX_NAME, query=query)
print(response.body)

client.end_transaction(name="search-auto-instrument2", result="success")

{'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


## サーチの実行 - 3 （無効なクエリー)

In [10]:
client.begin_transaction(transaction_type="app")
elasticapm.set_user_context(username="nobuhiko")

try:
  search_key_word="iPhone"
  query={"fuzzzzzzzzy": {"question": search_key_word}}
  response = es.search(index=INDEX_NAME, query=query)
  print(response.body)
except Exception as e:
  print(e)
  elasticapm.set_transaction_outcome("failure")
  client.capture_exception()
finally:
  client.end_transaction(name="search-auto-instrument3", result="failure")

BadRequestError(400, 'parsing_exception', 'unknown query [fuzzzzzzzzy]')
